### Import Required Libraries and Set Up Environment Variables

In [43]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
from pandas import json_normalize

In [45]:
# Set environment variables from the .env in the local environment
load_dotenv('example.env')

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [6]:
# Set the base URL
base_url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = (f"{base_url}"
       f"fq={filter_query}&"
       f"sort={sort}&"
       f"fl={field_list}&"
       f"begin_date={begin_date}&"
       f"end_date={end_date}&"
       f"api-key={nyt_api_key}")


In [8]:
query_url

'https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&begin_date=20130101&end_date=20230531&api-key=3QVubjH9ccKhAN6jUUn5rhsEPVaVeONJ'

In [38]:
# Create an empty list to store the reviews
reviews_list = []

# Loop through pages 0-19
for page in range(0, 20):
    # Create query with a page number
    # API results show 10 articles at a time
    query_url = f"{query_url}&page={str(page)}"
   
    # Make a "GET" request and retrieve the JSON
    try:
        response = requests.get(query_url)
        if response.status_code == 200:
            reviews = response.json()
            
            if "docs" not in reviews["response"] or len(reviews["response"]["docs"]) == 0:
                print(f"No results found on page {page}. Exiting the loop.")
                break
            
            # Try to save the reviews to the reviews_list
            # Loop through the reviews["response"]["docs"] and append each review to the list
            for review in reviews["response"]["docs"]:
                reviews_list.append(review)
                
            # Print the page number that was just retrieved
            print(f"Page {page} retrieved successfully.")
        
        # Add a twelve-second interval between queries to stay within API query limits
        time.sleep(12)

    except KeyError:
        print(f"KeyError: No results found on page {page}. Exiting the loop.")
        break
    except Exception as e:
        print(f"An error occurred on page {page}: {e}")
        break




Page 0 retrieved successfully.
Page 1 retrieved successfully.
Page 2 retrieved successfully.
Page 3 retrieved successfully.
Page 4 retrieved successfully.
Page 5 retrieved successfully.
Page 6 retrieved successfully.
Page 7 retrieved successfully.
Page 8 retrieved successfully.
Page 9 retrieved successfully.
Page 10 retrieved successfully.
Page 11 retrieved successfully.
Page 12 retrieved successfully.
Page 13 retrieved successfully.
Page 14 retrieved successfully.
Page 15 retrieved successfully.
Page 16 retrieved successfully.
Page 17 retrieved successfully.
Page 18 retrieved successfully.
Page 19 retrieved successfully.


In [48]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
preview_data = reviews_list[:5]
formatted_json = json.dumps(preview_data, indent=4)
print(formatted_json)

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [50]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
df = json_normalize(reviews_list)
df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
196,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
197,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
198,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None


In [56]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
df['title'] = df['headline.main'].apply(
    lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")] if isinstance(st, str) and "\u2018" in st and "\u2019 Review" in st else None)
df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves
196,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story
197,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One
198,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Your Place or Mine


In [58]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
def convert_keywords_to_string(keyword_list):
    if isinstance(keyword_list, list):
        return "; ".join([f"{item['name']}: {item['value']}" for item in keyword_list if 'name' in item and 'value' in item])
    else:
        return keyword_list
df['keywords_fixed'] = df['keywords'].apply(convert_keywords_to_string)
df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title,keywords_fixed
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,subject: Movies; creative_works: The Attachmen...
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,None,"subject: Movies; persons: Kapur, Shekhar; pers..."
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever,subject: Movies; creative_works: You Can Live ...
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love,subject: Movies; creative_works: A Tourist's G...
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children,"subject: Movies; persons: Zlotowski, Rebecca; ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves,"subject: Movies; persons: Bracey, Luke (1989- ..."
196,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story,subject: Documentary Films and Programs; creat...
197,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One,subject: Movies; creative_works: A Thousand an...
198,https://www.nytimes.com/2023/02/09/

In [62]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = df['title'].to_list()
titles

['The Attachment Diaries',
 None,
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'The Attachment Diaries',
 None,
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'The Attachment Diaries',
 None,
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'The Attachment Diaries',
 None,
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',


### Access The Movie Database API

In [68]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [82]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple of 50 requests
request_counter = 0

# Base URLs for TMDB API
search_url = "https://api.themoviedb.org/3/search/movie?query="
movie_details_url = "https://api.themoviedb.org/3/movie/"

# Loop through the titles
for title in titles_list:
    
    # Check if we need to sleep before making a request
    if request_counter > 0 and request_counter % 50 == 0:
        print(f"Reached {request_counter} requests. Sleeping for 1 second...")
        time.sleep(1)
    
    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    try:
        search_query_url = f"{search_url}{title}&api_key={tmdb_api_key}"
        search_response = requests.get(search_query_url)
        
        # Check if the request was successful
        if search_response.status_code == 200:
            search_data = search_response.json()
            
            # Check if results exist
            if search_data['results']:
                
                # Get movie id
                movie_id = search_data['results'][0]['id']
                
                # Make a request for the full movie details
                details_query_url = f"{movie_details_url}{movie_id}?api_key={tmdb_api_key}"
                details_response = requests.get(details_query_url)
                
                # Execute "GET" request with url
                if details_response.status_code == 200:
                    movie_details = details_response.json()
                    
                    # Extract the genre names into a list
                    genres = [genre['name'] for genre in movie_details.get('genres', [])]
                    
                    # Extract the spoken_languages' English name into a list
                    spoken_languages = [lang['english_name'] for lang in movie_details.get('spoken_languages', [])]
                    
                    # Extract the production_countries' name into a list
                    production_countries = [country['name'] for country in movie_details.get('production_countries', [])]
                    
                    # Add the relevant data to a dictionary and append it to the tmdb_movies_list list
                    movie_data = {
                        'title': movie_details.get('title', 'N/A'),
                        'release_date': movie_details.get('release_date', 'N/A'),
                        'genres': genres,
                        'spoken_languages': spoken_languages,
                        'production_countries': production_countries
                    }
                    tmdb_movies_list.append(movie_data)
                    
                    # Print out the title that was found
                    print(f"Found details for {movie_details.get('title', 'N/A')}")
                
                else:
                    print(f"Could not retrieve details for movie with ID {movie_id}")
            else:
                print(f"No results found for title: {title}")
        
    except Exception as e:
        # Use the except clause to print out a statement if a movie is not found
        print(f"An error occurred while processing the title '{title}': {str(e)}")

# Print out the number of movies found and details
print(f"Total movies found: {len(tmdb_movies_list)}")

Found details for The Attachment Diaries
Found details for Red Light Green Light
Found details for You Can Live Forever
Found details for A Tourist's Guide to Love
Found details for Other People's Children
Found details for One True Loves
Found details for The Lost Weekend: A Love Story
Found details for A Thousand and One
Found details for Your Place or Mine
Found details for Love in the Time of Fentanyl
Found details for The Attachment Diaries
Found details for Red Light Green Light
Found details for You Can Live Forever
Found details for A Tourist's Guide to Love
Found details for Other People's Children
Found details for One True Loves
Found details for The Lost Weekend: A Love Story
Found details for A Thousand and One
Found details for Your Place or Mine
Found details for Love in the Time of Fentanyl
Found details for The Attachment Diaries
Found details for Red Light Green Light
Found details for You Can Live Forever
Found details for A Tourist's Guide to Love
Found details for 

In [86]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
preview_data = tmdb_movies_list[:5]
formatted_json = json.dumps(preview_data, indent=4)

print(formatted_json)

[
    {
        "title": "The Attachment Diaries",
        "release_date": "2021-10-07",
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
    },
    {
        "title": "Red Light Green Light",
        "release_date": "",
        "genres": [
            "Science Fiction",
            "Horror"
        ],
        "spoken_languages": [
            "English"
        ],
        "production_countries": [
            "United States of America"
        ]
    },
    {
        "title": "You Can Live Forever",
        "release_date": "2023-03-24",
        "genres": [
            "Drama",
            "Romance"
        ],
        "spoken_languages": [
            "English",
            "French"
        ],
        "production_countries": [
            "Canada",
            "United States of America"
  

In [94]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df

,title,release_date,genres,spoken_languages,production_countries
0,The Attachment Diaries,2021-10-07,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,Red Light Green Light,,"[Science Fiction, Horror]",[English],[United States of America]
2,You Can Live Forever,2023-03-24,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
3,A Tourist's Guide to Love,2023-04-21,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
4,Other People's Children,2022-09-21,"[Drama, Comedy]","[French, English]",[France]
...,...,...,...,...,...
195,One True Loves,2023-04-07,"[Romance, Comedy, Drama]","[English, Spanish]","[Czech Republic, United States of America]"
196,The Lost Weekend: A Love Story,2023-04-13,[Documentary],[English],[United States of America]
197,A Thousand and One,2023-03-31,"[Drama, Crime]","[English, Spanish, Portuguese]",[United States of America]
198,Your Place or Mine,2023-02-10,"[Romance, Comedy]",[English],[United States of America]


### Merge and Clean the Data for Export

In [100]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(df, tmdb_df, on='title', how='inner')
merged_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,headline.sub,byline.original,byline.person,byline.organization,title,keywords_fixed,release_date,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,subject: Movies; creative_works: The Attachmen...,2021-10-07,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,subject: Movies; creative_works: The Attachmen...,2021-10-07,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
2,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,subject: Movies; creative_works: The Attachmen...,2021-10-07,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
3,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,subject: Movies; creative_works: The Attachmen...,2021-10-07,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
4,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,subject: Movies; creative_works: The Attachmen...,2021-10-07,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]


In [112]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ['[', ']', "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].apply(lambda x: ', '.join(x) if isinstance(x, list) else str(x))
    # Loop through characters to remove
    for char in characters_to_remove:
        merged_df[column] = merged_df[column].str.replace(char, '')
# Display the fixed DataFrame
merged_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,headline.seo,headline.sub,byline.original,byline.organization,title,keywords_fixed,release_date,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,None,By Jeannette Catsoulis,None,The Attachment Diaries,subject: Movies; creative_works: The Attachmen...,2021-10-07,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,None,By Jeannette Catsoulis,None,The Attachment Diaries,subject: Movies; creative_works: The Attachmen...,2021-10-07,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
2,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,None,By Jeannette Catsoulis,None,The Attachment Diaries,subject: Movies; creative_works: The Attachmen...,2021-10-07,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
3,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,None,By Jeannette Catsoulis,None,The Attachment Diaries,subject: Movies; creative_works: The Attachmen...,2021-10-07,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
4,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,None,By Jeannette Catsoulis,None,The Attachment Diaries,subject: Movies; creative_works: The Attachmen...,2021-10-07,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2795,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-02-02T12:00:11+0000,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,...,None,None,By Concepción de León,None,Love in the Time of Fentanyl,subject: Documentary Films and Programs; subje...,2023-02-03,Documentary,English,"Canada, United States of America"
2796,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-02-02T12:00:11+0000,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,...,None,None,By Concepción de León,None,Love in the Time of Fentanyl,subject: Documentary Films and Programs; subje...,2023-02-03,Documentary,English,"Canada, United States of America"
2797,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-02-02T12:00:11+0000,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,...,None,None,By Concepción de León,None,Love in the Time of Fentanyl,subjec

In [114]:
# Drop "byline.person" column
merged_df = merged_df.drop(columns=["byline.person"])

KeyError: "['byline.person'] not found in axis"

In [ ]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates()
merged_df = merged_df.reset_index(drop=True)
merged_df

In [117]:
# Export data to CSV without the index
merged_df.to_csv('merged_data.csv', index=False)